In [20]:
import numpy as np
import pandas as pd
from googleapiclient import discovery
import json
from time import sleep
from tqdm import tqdm

API_KEY = '[insert-api-key-here]'

In [16]:
test_ds = pd.read_csv("..//Data//PreprocessedData//test_preprocessed.csv")

In [17]:
test_ds = test_ds["preprocessed_text"]

In [28]:
def get_toxicity_scores(text):
    client = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=API_KEY,
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
    )

    analyze_request = {
    'comment': { 'text': text },
    'languages': ["en"],
    'requestedAttributes': {'TOXICITY': {}}
    }

    response = client.comments().analyze(body=analyze_request).execute()
    return int(response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]<0.5)

In [29]:
labels = []
for each in tqdm(test_ds.values):
    labels.append(get_toxicity_scores(each))
    sleep(1)

100%|██████████| 860/860 [22:41<00:00,  1.58s/it]


In [30]:
df = pd.DataFrame({"id": range(len(labels)), "label": labels})

In [31]:
df.head()

,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [32]:
df.to_csv("..//Results//perspective_2.csv", index = False)